In [1]:
import MetaTrader5 as mt5
import pandas as pd
from time import sleep

In [13]:
def market_order(symbol, volume, order_type, deviation=20, magic=30, stoploss=0.0, take_profit=0.0,
                 strategy_name='Candlestick_Engulfing_Bot2'):
    order_type_dict = {
        'buy': mt5.ORDER_TYPE_BUY,
        'sell': mt5.ORDER_TYPE_SELL
    }

    price_dict = {
        'buy': mt5.symbol_info_tick(symbol).ask,
        'sell': mt5.symbol_info_tick(symbol).bid
    }

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": volume,  # FLOAT
        "type": order_type_dict[order_type],
        "price": price_dict[order_type],
        "sl": stoploss,  # FLOAT
        "tp": take_profit,  # FLOAT
        "deviation": deviation,  # INTERGER
        "magic": magic,  # INTERGER
        "comment": strategy_name,
        "type_time": mt5.ORDER_TIME_GTC,  #The order stays in the queue until it is manually canceled
        "type_filling": mt5.ORDER_FILLING_FOK,  #This execution policy means that an order can be executed only in the specified volume
    }

    order_result = mt5.order_send(request)
    return (order_result)


In [14]:
def find_engulfing_pattern(candle):
    bull_cond1 = candle['close'] > candle['open']  # bull candle condition
    bull_cond2 = candle['close'] > candle['previous_high']  # engulfment condition
    bull_cond3 = candle['previous_open'] > candle['previous_close']  # previous candle must be a bear candle

    bear_cond1 = candle['close'] < candle['open']  # bear candle condition
    bear_cond2 = candle['close'] < candle['previous_low']  # engulfment condition
    bear_cond3 = candle['previous_open'] < candle['previous_close']  # previous candle must be a bull candle

    # special condition - engulfing candle body is 1.5 times as long as previous candle range
    if candle['previous_high'] - candle['previous_low'] == 0:
        return False

    special_cond = abs(candle['open'] - candle['close']) / (candle['previous_high'] - candle['previous_low']) >= 1.5

    if bull_cond1 and bull_cond2 and bull_cond3 and special_cond:
        return 'buy'
    elif bear_cond1 and bear_cond2 and bear_cond3 and special_cond:
        return 'sell'
    else:
        return False

In [15]:
#It's find out the bullish and bearish 

def get_engulfing_signal(symbol, timeframe):
    ohlc = mt5.copy_rates_from_pos(symbol, timeframe, 1, 2)  #initial bar, numner of bars
    
    ohlc_df = pd.DataFrame(ohlc)[['time', 'open', 'high', 'low', 'close']]
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit='s') #convert the in real datetime frame

    ohlc_df['previous_open'] = ohlc_df['open'].shift(1)
    ohlc_df['previous_high'] = ohlc_df['high'].shift(1)
    ohlc_df['previous_low'] = ohlc_df['low'].shift(1)
    ohlc_df['previous_close'] = ohlc_df['close'].shift(1)

    ohlc_df['signal'] = ohlc_df.apply(find_engulfing_pattern, axis=1)

    print(ohlc_df)
    return ohlc_df.iloc[-1]['signal']

In [16]:
if __name__ == '__main__':
    
    mt5.initialize()
    login = ********
    password = "*******"
    server = "MetaQuotes-Demo"
    mt5.login(login, password, server)

    # strategy parameters
    symbol = 'EURUSD'
    timeframe = mt5.TIMEFRAME_M1
    volume = 0.01

    trading_allowed = True
    while trading_allowed:
        # Careful! Loop can open infinite positions!

        signal = get_engulfing_signal(symbol, timeframe) #define the buy/sell
        print('signal', signal)
        print('---\n')

        if signal == 'buy' and mt5.positions_total() == 0:
            res = market_order(symbol, volume, 'buy')
            trading_allowed = False

        elif signal == 'sell' and mt5.positions_total() == 0:
            res = market_order(symbol, volume, 'sell')
            trading_allowed = False

        sleep(1)
        
        

                 time     open     high      low    close  previous_open  \
0 2024-02-13 14:14:00  1.07811  1.07814  1.07807  1.07813            NaN   
1 2024-02-13 14:15:00  1.07814  1.07830  1.07814  1.07818        1.07811   

   previous_high  previous_low  previous_close  signal  
0            NaN           NaN             NaN   False  
1        1.07814       1.07807         1.07813   False  
signal False
---

                 time     open     high      low    close  previous_open  \
0 2024-02-13 14:21:00  1.07781  1.07781  1.07763  1.07771            NaN   
1 2024-02-13 14:22:00  1.07771  1.07790  1.07767  1.07790        1.07781   

   previous_high  previous_low  previous_close  signal  
0            NaN           NaN             NaN   False  
1        1.07781       1.07763         1.07771   False  
signal False
---

                 time     open     high      low    close  previous_open  \
0 2024-02-13 14:21:00  1.07781  1.07781  1.07763  1.07771            NaN   
1 2024-02-13

                 time     open    high      low    close  previous_open  \
0 2024-02-13 14:22:00  1.07771  1.0779  1.07767  1.07790            NaN   
1 2024-02-13 14:23:00  1.07786  1.0779  1.07781  1.07788        1.07771   

   previous_high  previous_low  previous_close  signal  
0            NaN           NaN             NaN   False  
1         1.0779       1.07767          1.0779   False  
signal False
---

                 time     open    high      low    close  previous_open  \
0 2024-02-13 14:22:00  1.07771  1.0779  1.07767  1.07790            NaN   
1 2024-02-13 14:23:00  1.07786  1.0779  1.07781  1.07788        1.07771   

   previous_high  previous_low  previous_close  signal  
0            NaN           NaN             NaN   False  
1         1.0779       1.07767          1.0779   False  
signal False
---

                 time     open    high      low    close  previous_open  \
0 2024-02-13 14:22:00  1.07771  1.0779  1.07767  1.07790            NaN   
1 2024-02-13 14:23:0

KeyboardInterrupt: 